<div style="background-color: lightblue; padding: 30px;">
    
# Digital House - Data Science a Distancia

## Proyecto Final Integrador

### Autores: Daniel Borrino, Ivan Mongi, Jessica Polakoff, Julio Tentor
    
<p style="text-align:right;">Agosto 2022</p>

---
</div>    

---
## Objetivo del Trabajo Final Integrador

<h4 style="text-align:center;">Demostrar que aprendimos conceptos y desarrollamos habilidades para obtener un modelo de aprendizaje automático que razonablemente predice un valor</h4>

---
### Contexto

???
---

---
### Requisitos del Trabajo Final Integrador

- Utilizar modelos de regresión o clasificación (es opcional)
- Evaluar modelos mediante ensemble, bagging o boosting
- Realizar selección de hiperparámetros
- Implementar pipeline al menos con algunos pasos del preprocesamiento
- Analizar importancia de características
- Serializar el modelo con mejor desempeño

---
### Entregables del Trabajo Final Integrador
#### Técnico (Jupyter Notebook)

1. Introducción y planteo del problema
2. Técnicas a utilizar y detalle del conjunto de datos
3. Desarrollo del análisis, visualizaciones, resultados de modelos
4. Detalle de hallazgos, conclusiones y recomendaciones

#### Presentación (audiencia NO TÉCNICA)

1. Resumen del problema
2. Métodos utilizados
3. Principales resultados

---
### Aspectos técnicos

La notebook se ejecuta correctamente en una instalación estándar de Anaconda versión *4.13.0 build  3.21.6, Python 3.9.7*

También hacen falta las siguientes librerías:

- [XGBoost](https://xgboost.readthedocs.io/en/stable/install.html)


---


---

---
<a id="NecessaryLibraries"></a>

#### Necessary libraries


In [1]:
# imprescindible
import pandas as pd
import numpy as np

# to avoid some warnings messages
import warnings
warnings.filterwarnings('ignore')

# to draw some graphs
import seaborn as sns
import matplotlib.pyplot as plt

# set seaborn and matplotlib default theme
sns.set_theme()
_sns_plotting_contex_ = sns.plotting_context()
sns.plotting_context('poster')

# set seaborn and matplotlib style to ...
# plt.style.use('classic')
sns.mpl.rcParams['axes.titlesize'] = 18
sns.mpl.rcParams['axes.labelsize'] = 14

# to use HTML codes within IPpython.display function
from IPython.display import HTML



### ???

---
<a id="UsefulFunctions"></a>

### Funciones útiles


In [19]:
def blogData_train_read() :
    u''' Reads and prepare data from blog feedback data train set
    
    '''

    data = pd.read_csv("./data/blogData_train.csv", header=None)
    data.drop_duplicates(inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    header = pd.read_csv("./data/blogData_label.csv", header=None)
    header = list(header[0])
    
    if len(header) != data.shape[1] :
        raise Exception('Los encabezados y la cantidad de características NO COINCIDE !!!')

    data.columns = header
    
    return data


In [ ]:
def blogData_features(data) :
    u''' Create a dictionary with some keys associates to list of features in the final work dataframe
    
    args:
        data : DataFrame
    
    return:
        dictionary with util features names
    
    '''
    columns = list(data.columns)

    features = dict()

    features['sd_nc_total_before_BT'] = columns[0:5]
    features['sd_nc_24_before_BT'] = columns[5:10]
    features['sd_nc_between_24_48'] = columns[10:15]
    features['sd_nc_first_24_BT'] = columns[15:20]
    features['sd_nc_diff_24_48'] = columns[20:25]
    
    features['sd_nl_total_before_BT'] = columns[25:30]
    features['sd_nl_24_before_BT'] = columns[30:35]
    features['sd_nl_between_24_48'] = columns[35:40]
    features['sd_nl_first_24_BT'] = columns[40:45]
    features['sd_nl_diff_24_48'] = columns[45:50]
    
    features['nc_total_before_BT'] = columns[50:51]
    features['nc_24_before_BT'] = columns[51:52]
    features['nc_between_24_48'] = columns[52:53]
    features['nc_first_24_BT'] = columns[53:54]
    features['nc_diff_24_48'] = columns[54:55]
    
    features['nl_total_before_BT'] = columns[55:56]
    features['nl_24_before_BT'] = columns[56:57]
    features['nl_between_24_48'] = columns[57:58]
    features['nl_first_24_BT'] = columns[58:59]
    features['nl_diff_24_48'] = columns[59:60]
    
    features['nc'] = columns[50:55]
    features['nl'] = columns[55:60]

    features['timelength_post_BT'] = columns[60:61]
    features['length_post'] = columns[61:62]

    features['frequent_word'] = columns[62:262]

    features['weekday_BT'] = columns[262:269]
    features['weekday_post'] = columns[269:276]
    features['parents'] = columns[276:280]
    features['comments'] = columns[280:281]

    return features


In [8]:


def set_figure(row, col, suptitle=None) :
    u''' Activate matplot figure setting size and super title
    '''
    fig = plt.figure(figsize=(row, col));
    if suptitle != None :
        fig.suptitle(suptitle, 
                     verticalalignment='center', fontsize='xx-large', fontweight='extra bold');
    return fig


def show_corr(data, target) :
    u'''
    '''
    
    corr = data.corr()
    cols = corr[target].sort_values(ascending=False).keys()
    
    topc = corr.loc[cols, cols]
    mask = np.zeros_like(topc)
    mask[np.triu_indices_from(mask)] = True
    
    sns.heatmap(data=topc, 
                cmap=sns.diverging_palette(h_neg=350, h_pos=150, center='light', as_cmap=True), 
                center=0, linewidths=1, annot=True, fmt=".3f", cbar=False, mask=mask)

    return

---



---
<a id="DatasetDescription"></a>

### Descripción del Dataset

Los datos originales se obtuvieron de [Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/BlogFeedback) que corresponden a la publicación de Kristian Buza (2012) sobre [Feedback Prediction for Blogs](http://cs.bme.hu/~buza/pdfs/gfkl2012_blogs.pdf)

---

El conjunto de datos se compone de 281 características y 49203 observaciones útiles, no duplicadas

---
*Las características son*:

- 0..49: Media, desviación estándar, mínimo, máximo y mediana de cada una de las características 50 a 59 del origen o sitio de las publicaciones. Se entiende que origen o sitio es la página web en la que se encuentra la publicación
- 50: Número total de comentarios antes de la fecha límite
- 51: Número de comentarios en las últimas 24 horas antes de la fecha límite
- 52: Número de comentarios en el período de tiempo transcurrido entre las 48 y 24 horas antes de la fecha límite
- 53: Número de comentarios en las primeras 24 horas después de realizar la publicación pero antes de de la fecha límite
- 54: Diferencia entre las características 51 y 52
- 55…59: Las mismas características indicadas para 50..54 pero referidas al número de enlaces (trackbacks)
- 60: Cantidad de “tiempo” entre el momento en que se realizó la publicación y la fecha límite
- 61: Longitud del texto de la publicación
- 62…261: Indicador binario para las 200 palabras más frecuentes en el texto de la publicación
- 262…268: Indicador binario del día de la semana (lunes a domingo) de la fecha límite
- 269…275: Indicador binario del día de la semana (lunes a domingo) de la fecha de publicación
- 276: Número de páginas padres, se considera página padre cuando la publicación es una réplica (trackback) de ella
- 277…279: Mínimo, máximo y media de comentarios en la página padre
- 280: Número de comentarios en las siguientes 24 horas de la fecha límite

---


---
<a id="ExploratoryDataAnalysis_1"></a>

### Exploración y Análisis de Datos Básico 
---


---
#### Observaciones

- ???


---

---

---

---
### Evaluate base line models
---

---
#### Observaciones *algo para decir ...*

- ???

---

---

---



---
